In [81]:
import pandas as pd
import requests
import json
import datetime as dt
import os
import plotly as plt
import tensorflow as tf
import keras
import numpy as np
import pyod
from pyod.models.ecod import ECOD
import time
import plotly.express as px

# Generation Data

In [479]:
json_file_path = 'C:/Users/EMC/Documents/doutorado/doctorade/data/loanda/generation_data/'

os.listdir(json_file_path)

['loanda_data_curated_0.json',
 'loanda_data_curated_200000.json',
 'loanda_data_curated_400000.json',
 'loanda_data_curated_600000.json',
 'loanda_data_curated_800000.json']

In [136]:
json_file_path = 'C:/Users/EMC/Documents/doutorado/doctorade/data/loanda/generation_data/'
df_generation = pd.DataFrame()
for data_chunks in os.listdir(json_file_path):
    path = json_file_path + data_chunks
    with open(path, "r") as json_file:
    # Load the JSON data into a Python object.
        json_data = json.load(json_file)
        pd_json_data = pd.DataFrame.from_dict(json_data)
        df_generation = pd.concat([df_generation,pd_json_data])

df = df_generation
# Find None
df = df.replace('nan', None)
# To Datetime
df['E3TimeStamp'] = pd.to_datetime(df['E3TimeStamp'], format = 'mixed')

df_time = df['E3TimeStamp']

df_float = df[['PotenciaAtivaTotal', 'EnergiaTotal_DataLogger',
       'EnergiaDiaria_DataLogger', 'RadiacaoGlobal', 'TempAr', 'TempModulo',
       'VelocidadeVento', 'EnergiaTotal_LoteB', 'EnergiaDiaria_LoteB',
       'EnergiaReativa_LoteB', 'EnergiaTotal_LoteC', 'EnergiaDiaria_LoteC',
       'EnergiaReativa_LoteC', 'EnergiaTotal_LoteD', 'EnergiaDiaria_LoteD',
       'EnergiaReativa_LoteD', 'EnergiaTotal_LoteE', 'EnergiaDiaria_LoteE',
       'EnergiaReativa_LoteE', 'EnergiaTotal_LoteF', 'EnergiaDiaria_LoteF',
       'EnergiaReativa_LoteF']].astype(float)

df = pd.concat([df_time,df_float], axis = 1)

df = df.sort_values(by='E3TimeStamp')

# Criar geração por minuto
try:
    df = df.set_index('E3TimeStamp')
except:
    pass

# Diferenças negativas
df['yB'] = df.EnergiaDiaria_LoteB.diff().shift(-1)
df['yC'] = df.EnergiaDiaria_LoteC.diff().shift(-1)
df['yD'] = df.EnergiaDiaria_LoteD.diff().shift(-1)
df['yE'] = df.EnergiaDiaria_LoteE.diff().shift(-1)
df['yF'] = df.EnergiaDiaria_LoteF.diff().shift(-1)

# Excluir valores nulos pelo Lote F onde temos o início mais tardio.

df = df[~(df['yF'].isna())]

# Energia gerada por minuto

In [113]:
df_teste = df[(df.index >= '2022-01-27') & (df.index < '2022-01-28')]
plt.plot(x = df_teste.index,y= [df_teste['yF']], data_frame=df_teste, kind='line')

# Verificar correlation entre as variáveis

In [87]:
# Correlation
df_corr = df[['RadiacaoGlobal', 'TempAr', 'TempModulo',
       'VelocidadeVento', 'yB', 'yC', 'yD', 'yE', 'yF']].corr().round(2)
fig = px.imshow(df_corr, text_auto=True, aspect="auto")
fig.show()

# Tabela resumida dos conteúdo de Luanda.

In [4]:
try:
  df = df.set_index('E3TimeStamp')
except:
  pass


df_filtered = df[['RadiacaoGlobal', 'TempAr', 'TempModulo',
       'VelocidadeVento', 'yB', 'yC', 'yD', 'yE', 'yF']]

df_descricao = pd.DataFrame()

df_descricao['No_None_Values_Count'] = round(df_filtered.apply(lambda x: x.count()),2)
df_descricao['None_Values_Count'] = round(df_filtered.apply(lambda x: x.isnull().sum()),2)
df_descricao['Zeroes_Values_Count'] = round(df_filtered.apply(lambda x: (x == 0).sum()),2)
df_descricao['Min_Values'] = round(df_filtered.apply(lambda x: x.min()),2)
df_descricao['Max_Values'] = round(df_filtered.apply(lambda x: x.max()),2)
df_descricao['Mean_Values'] = round(df_filtered.apply(lambda x: x.mean()),2)
df_descricao['Std_Values'] = round(df_filtered.apply(lambda x: x.std()),2)
df_descricao['Sum of Values'] = round(df_filtered.apply(lambda x: x.sum()),0)
df_descricao['Min_Date'] = df_filtered.index.min()
df_descricao['Max_Date'] = df_filtered.index.max()
df_descricao

,No_None_Values_Count,None_Values_Count,Zeroes_Values_Count,Min_Values,Max_Values,Mean_Values,Std_Values,Sum of Values,Min_Date,Max_Date
RadiacaoGlobal,772953,0,33797,-20.0,1650.0,215.72,321.27,166744947.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
TempAr,772953,0,6502,0.0,195.0,23.51,7.07,18175966.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
TempModulo,772953,0,6502,0.0,197.0,27.74,14.18,21439342.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
VelocidadeVento,772953,0,145677,-10.0,58.0,0.50,4.16,383587.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
yB,772953,0,428786,-8632.2,8027.3,0.01,203.39,5570.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
yC,772953,0,434853,-8636.8,8334.3,0.01,207.56,5531.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
yD,772953,0,447550,-8661.7,8338.8,0.01,201.44,5543.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
yE,772953,0,428434,-8705.4,8170.8,0.01,172.83,5488.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
yF,772953,0,422654,-8642.1,7689.6,0.01,172.34,5634.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373


# Anomaly detection

## PYOD Unsupervised Outlier Detection Using Empirical Cumulative Distribution Functions <br>

In [183]:
start_time = time.time()

df_anomaly = df[['yB', 'yC', 'yD', 'yE', 'yF','VelocidadeVento','TempModulo','TempAr','RadiacaoGlobal']].copy()

train_row_number = 80000

for y in df_anomaly.columns:
    df_y = df_anomaly[[y]]

    # Normalize and save the mean and std we get,
    # for normalizing test data.
    training_mean = df_y.mean()
    training_std = df_y.std()
    df_y_normalized = (df_y - training_mean) / training_std

    train = df_y_normalized.iloc[:train_row_number,:]
    test = df_y_normalized.iloc[train_row_number:,:]

    clf = ECOD()
    clf.fit(train)

    # get outlier scores
    y_train_scores = clf.decision_scores_  # raw outlier scores on the train data
    y_test_scores = clf.decision_function(test)  # predict raw outlier scores on test

    # The threshold is 6 times the standard deviation
    results_ecod_test = df_y[[y]].iloc[train_row_number:,:][(y_test_scores > y_test_scores.std()*6)]
    results_ecod_training = df_y[[y]].iloc[:train_row_number,:][(y_train_scores > y_train_scores.std()*6)]

    total_anomalias = df_anomaly.loc[pd.concat([results_ecod_test,results_ecod_training]).index,:]

    df_y.loc[total_anomalias.index,:] = None
    df_anomaly[str(y) + '_anomalia'] = False
    df_anomaly.loc[total_anomalias.index,str(y) + '_anomalia'] = True

    df_anomaly[y] = df_y[y].interpolate()

end_time = time.time()

# Resultado ECOD:
ecod_results = pd.DataFrame(df_anomaly[['yB_anomalia', 'yC_anomalia', 'yD_anomalia',
   'yE_anomalia', 'yF_anomalia', 'VelocidadeVento_anomalia',
   'TempModulo_anomalia', 'TempAr_anomalia',
   'RadiacaoGlobal_anomalia']].sum()/len(df_anomaly) * 100, columns = ["% of anomaly"])
ecod_results.loc['algoritmo_anomalia',:] = 'PyOD - ECOD'
ecod_results.loc['tempo_execucao (s)',:] = end_time - start_time
ecod_results.loc['threshold',:] = "6x scores std"
ecod_results.loc['% of traning samples',:] = len(train)/len(df_anomaly)*100

## Resultado ECOD
- % Anomalia<br>
- Verificação dos valores máximos e mínimos
- Correlação entre variáveis
- Threshold utilizado

In [185]:
ecod_results

,% of anomaly
yB_anomalia,0.525776
yC_anomalia,0.509087
yD_anomalia,0.473379
yE_anomalia,0.546864
yF_anomalia,0.504688
VelocidadeVento_anomalia,0.475449
TempModulo_anomalia,0.250339
TempAr_anomalia,0.251891
RadiacaoGlobal_anomalia,0.59111
algoritmo_anomalia,PyOD - ECOD


In [72]:
try:
  df = df.set_index('E3TimeStamp')
except:
  pass


df_filtered = df_anomaly[['RadiacaoGlobal', 'TempAr', 'TempModulo',
       'VelocidadeVento', 'yB', 'yC', 'yD', 'yE', 'yF']]

df_descricao = pd.DataFrame()

df_descricao['No_None_Values_Count'] = round(df_filtered.apply(lambda x: x.count()),2)
df_descricao['None_Values_Count'] = round(df_filtered.apply(lambda x: x.isnull().sum()),2)
df_descricao['Zeroes_Values_Count'] = round(df_filtered.apply(lambda x: (x == 0).sum()),2)
df_descricao['Min_Values'] = round(df_filtered.apply(lambda x: x.min()),2)
df_descricao['Max_Values'] = round(df_filtered.apply(lambda x: x.max()),2)
df_descricao['Mean_Values'] = round(df_filtered.apply(lambda x: x.mean()),2)
df_descricao['Std_Values'] = round(df_filtered.apply(lambda x: x.std()),2)
df_descricao['Sum of Values'] = round(df_filtered.apply(lambda x: x.sum()),0)
df_descricao['Min_Date'] = df_filtered.index.min()
df_descricao['Max_Date'] = df_filtered.index.max()
df_descricao

,No_None_Values_Count,None_Values_Count,Zeroes_Values_Count,Min_Values,Max_Values,Mean_Values,Std_Values,Sum of Values,Min_Date,Max_Date
RadiacaoGlobal,772953,0,33797,-13.0,1324.0,214.70,318.62,165952794.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
TempAr,772953,0,6502,0.0,39.0,23.38,5.88,18073734.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
TempModulo,772953,0,6517,0.0,69.0,27.63,13.67,21357582.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
VelocidadeVento,772953,0,145683,-10.0,6.0,0.48,4.14,374234.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
yB,772953,0,429956,0.0,26.1,3.84,5.78,2971977.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
yC,772953,0,436247,0.0,34.6,3.74,5.70,2889621.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
yD,772953,0,448630,0.0,18.6,3.53,5.58,2730929.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
yE,772953,0,429668,0.0,17.2,3.79,5.71,2927693.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373
yF,772953,0,423469,0.0,17.2,3.88,5.79,2997650.0,2021-12-09 10:36:17.677,2023-08-09 22:19:10.373


In [86]:
df_corr = df_filtered.corr()

fig = px.imshow(df_corr, text_auto=True, aspect="auto")
fig.show()

# Criar granularidade de HORA

In [93]:
df_hora_soma = df_filtered[['yB','yC', 'yD', 'yE', 'yF']]
df_hora_soma = df_hora_soma.resample('H').sum()
df_hora_soma

,yB,yC,yD,yE,yF
E3TimeStamp,,,,,
2021-12-09 10:00:00,0.000000,0.000000,0.000000,0.0,0.0
2021-12-09 11:00:00,496.200000,486.650000,395.000000,0.0,0.0
2021-12-09 12:00:00,70.000000,71.800000,71.000000,0.0,0.0
2021-12-09 13:00:00,19.000000,86.000000,89.000000,0.0,0.0
2021-12-09 14:00:00,1433.030928,1540.227835,1601.291753,0.0,0.0
...,...,...,...,...,...
2023-08-09 18:00:00,2.100000,2.200000,2.300000,2.2,2.4
2023-08-09 19:00:00,0.000000,0.000000,0.000000,0.0,0.0
2023-08-09 20:00:00,0.000000,0.000000,0.000000,0.0,0.0


In [127]:
df_hora_media = df_filtered[['RadiacaoGlobal','TempAr', 'TempModulo','VelocidadeVento']]
df_hora_media = df_hora_media.resample('H').mean()
df_hora_media = df_hora_media.fillna(0)

In [128]:
df_hora = pd.concat([df_hora_soma,df_hora_media], axis = 1)
df_hora

,yB,yC,yD,yE,yF,RadiacaoGlobal,TempAr,TempModulo,VelocidadeVento
E3TimeStamp,,,,,,,,,
2021-12-09 10:00:00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2021-12-09 11:00:00,496.200000,486.650000,395.000000,0.0,0.0,237.593750,6.239583,13.682292,0.406250
2021-12-09 12:00:00,70.000000,71.800000,71.000000,0.0,0.0,441.411392,11.468354,25.405063,0.949367
2021-12-09 13:00:00,19.000000,86.000000,89.000000,0.0,0.0,474.673913,13.862319,21.539855,0.905797
2021-12-09 14:00:00,1433.030928,1540.227835,1601.291753,0.0,0.0,492.442478,16.805310,32.309735,1.274336
...,...,...,...,...,...,...,...,...,...
2023-08-09 18:00:00,2.100000,2.200000,2.300000,2.2,2.4,-1.833333,24.383333,22.200000,0.666667
2023-08-09 19:00:00,0.000000,0.000000,0.000000,0.0,0.0,-7.033333,22.566667,19.750000,0.483333
2023-08-09 20:00:00,0.000000,0.000000,0.000000,0.0,0.0,-4.483333,21.100000,18.066667,0.400000


In [130]:
df_corr = df_hora.corr()

fig = px.imshow(df_corr, text_auto=True, aspect="auto")
fig.show()

## Analise resultados com correlação e tabela de valores min, max, etc

In [129]:
try:
  df_hora = df_hora.set_index('E3TimeStamp')
except:
  pass

df_descricao = pd.DataFrame()

df_descricao['No_None_Values_Count'] = round(df_hora.apply(lambda x: x.count()),2)
df_descricao['None_Values_Count'] = round(df_hora.apply(lambda x: x.isnull().sum()),2)
df_descricao['Zeroes_Values_Count'] = round(df_hora.apply(lambda x: (x == 0).sum()),2)
df_descricao['Min_Values'] = round(df_hora.apply(lambda x: x.min()),2)
df_descricao['Max_Values'] = round(df_hora.apply(lambda x: x.max()),2)
df_descricao['Mean_Values'] = round(df_hora.apply(lambda x: x.mean()),2)
df_descricao['Std_Values'] = round(df_hora.apply(lambda x: x.std()),2)
df_descricao['Sum of Values'] = round(df_hora.apply(lambda x: x.sum()),0)
df_descricao['Min_Date'] = df_hora.index.min()
df_descricao['Max_Date'] = df_hora.index.max()
df_descricao

,No_None_Values_Count,None_Values_Count,Zeroes_Values_Count,Min_Values,Max_Values,Mean_Values,Std_Values,Sum of Values,Min_Date,Max_Date
yB,14605,0,8130,0.00,1433.03,203.49,322.09,2971977.0,2021-12-09 10:00:00,2023-08-09 22:00:00
yC,14605,0,8200,0.00,1540.23,197.85,317.00,2889621.0,2021-12-09 10:00:00,2023-08-09 22:00:00
yD,14605,0,8510,0.00,1601.29,186.99,311.82,2730929.0,2021-12-09 10:00:00,2023-08-09 22:00:00
yE,14605,0,8080,0.00,1008.10,200.46,318.80,2927693.0,2021-12-09 10:00:00,2023-08-09 22:00:00
yF,14605,0,8063,0.00,1008.20,205.25,324.06,2997650.0,2021-12-09 10:00:00,2023-08-09 22:00:00
RadiacaoGlobal,14605,0,1826,-11.14,1189.65,189.60,294.84,2769102.0,2021-12-09 10:00:00,2023-08-09 22:00:00
TempAr,14605,0,1806,0.00,38.78,20.65,9.30,301521.0,2021-12-09 10:00:00,2023-08-09 22:00:00
TempModulo,14605,0,1806,0.00,68.37,24.40,15.47,356310.0,2021-12-09 10:00:00,2023-08-09 22:00:00
VelocidadeVento,14605,0,1971,-10.00,5.51,0.43,3.84,6252.0,2021-12-09 10:00:00,2023-08-09 22:00:00


# Weather Data

In [131]:
path = 'C:/Users/EMC/Documents/doutorado/doctorade/data/loanda/weather_data/loanda_weather_curated.json'
with open(path, "r") as json_file:
# Load the JSON data into a Python object.
    json_data = json.load(json_file)
    df_weather = pd.DataFrame.from_dict(json_data)

df_time = df_weather['time_in_iso8601']

df_float = df_weather[[
       'temperature_2m_in_°C', 'relativehumidity_2m_in_%',
       'precipitation_in_mm', 'weathercode_in_wmo code_x', 'cloudcover_in_%',
       'cloudcover_low_in_%', 'cloudcover_mid_in_%', 'cloudcover_high_in_%',
       'windspeed_10m_in_km/h', 'is_day_in_', 'shortwave_radiation_in_W/m²',
       'direct_radiation_in_W/m²', 'diffuse_radiation_in_W/m²',
       'direct_normal_irradiance_in_W/m²', 'terrestrial_radiation_in_W/m²',
       'shortwave_radiation_instant_in_W/m²',
       'direct_radiation_instant_in_W/m²', 'diffuse_radiation_instant_in_W/m²',
       'direct_normal_irradiance_instant_in_W/m²',
       'terrestrial_radiation_instant_in_W/m²',
       'weathercode_in_wmo code_y', 'temperature_2m_max_in_°C',
       'temperature_2m_min_in_°C', 'temperature_2m_mean_in_°C',
       'apparent_temperature_max_in_°C', 'apparent_temperature_min_in_°C',
       'apparent_temperature_mean_in_°C', 'precipitation_sum_in_mm', 'rain_sum_in_mm',
       'precipitation_hours_in_h', 'windspeed_10m_max_in_km/h',
       'windgusts_10m_max_in_km/h', 'winddirection_10m_dominant_in_°',
       'shortwave_radiation_sum_in_MJ/m²', 'et0_fao_evapotranspiration_in_mm',
       'time_of_sunlight_in_iso8601']].astype(float)

df_weather = pd.concat([df_time,df_float], axis = 1)

df_weather = df_weather.sort_values(by='time_in_iso8601')

# Concat weather e generation data

In [132]:
df_hora = df_hora.reset_index()
df_hora['time_join_generation'] = df_hora['E3TimeStamp'].dt.date.astype(str) +' '+ df_hora['E3TimeStamp'].dt.hour.astype(str)
df_weather['time_in_iso8601'] = pd.to_datetime(df_weather['time_in_iso8601'], format='ISO8601')
df_weather['time_join_generation'] = df_weather['time_in_iso8601'].dt.date.astype(str) +' '+ df_weather['time_in_iso8601'].dt.hour.astype(str)

In [151]:
df_hora_all = df_hora.merge(df_weather,how='left',on = 'time_join_generation')
try:
    del df_hora_all['time_in_iso8601']
except:
    pass
try:
    del df_hora_all['time_join_generation']
except:
    pass
df_hora_all = df_hora_all.set_index("E3TimeStamp")


In [180]:
df_hora_all[df_hora_all['cloudcover_in_%'] > 90]

,yB,yC,yD,yE,yF,RadiacaoGlobal,TempAr,TempModulo,VelocidadeVento,temperature_2m_in_°C,...,apparent_temperature_mean_in_°C,precipitation_sum_in_mm,rain_sum_in_mm,precipitation_hours_in_h,windspeed_10m_max_in_km/h,windgusts_10m_max_in_km/h,winddirection_10m_dominant_in_°,shortwave_radiation_sum_in_MJ/m²,et0_fao_evapotranspiration_in_mm,time_of_sunlight_in_iso8601
E3TimeStamp,,,,,,,,,,,,,,,,,,,,,
2021-12-13 22:00:00,0.0,0.0,0.0,0.0,0.0,-3.183333,25.00,23.116667,0.300000,26.8,...,30.6,0.0,0.0,0.0,13.0,37.1,37.0,21.88,5.47,13.516667
2021-12-13 23:00:00,0.0,0.0,0.0,0.0,0.0,-2.950000,25.00,23.416667,0.250000,26.2,...,30.6,0.0,0.0,0.0,13.0,37.1,37.0,21.88,5.47,13.516667
2021-12-14 00:00:00,0.0,0.0,0.0,0.0,0.0,-3.500000,24.00,23.133333,0.850000,25.5,...,29.9,8.6,8.6,18.0,9.2,21.2,31.0,14.17,3.11,13.533333
2021-12-14 05:00:00,6.4,6.6,6.3,6.6,6.7,4.350000,24.00,23.483333,0.883333,22.6,...,29.9,8.6,8.6,18.0,9.2,21.2,31.0,14.17,3.11,13.533333
2021-12-14 06:00:00,61.6,61.0,61.5,61.9,60.6,53.300000,24.05,24.966667,1.516667,22.6,...,29.9,8.6,8.6,18.0,9.2,21.2,31.0,14.17,3.11,13.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-29 17:00:00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,23.3,...,21.2,0.0,0.0,0.0,9.2,21.6,140.0,10.08,2.06,11.033333
2023-07-29 18:00:00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,22.7,...,21.2,0.0,0.0,0.0,9.2,21.6,140.0,10.08,2.06,11.033333
2023-07-29 19:00:00,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,22.0,...,21.2,0.0,0.0,0.0,9.2,21.6,140.0,10.08,2.06,11.033333


In [156]:
df_corr = df_hora_all.corr()

In [173]:
df_corr.yB.sort_values()

relativehumidity_2m_in_%                   -0.348466
cloudcover_mid_in_%                        -0.128932
cloudcover_high_in_%                       -0.125020
precipitation_hours_in_h                   -0.109124
precipitation_sum_in_mm                    -0.098126
rain_sum_in_mm                             -0.098126
cloudcover_in_%                            -0.080739
weathercode_in_wmo code_y                  -0.054488
precipitation_in_mm                        -0.045192
windgusts_10m_max_in_km/h                  -0.018751
windspeed_10m_max_in_km/h                  -0.013998
weathercode_in_wmo code_x                  -0.005838
cloudcover_low_in_%                         0.010610
winddirection_10m_dominant_in_°             0.020030
apparent_temperature_min_in_°C              0.061281
temperature_2m_min_in_°C                    0.071677
apparent_temperature_mean_in_°C             0.096935
temperature_2m_mean_in_°C                   0.116408
apparent_temperature_max_in_°C              0.

# EDA

In [446]:
zeros_and_nulls = pd.concat([round((df == 0).sum()/len(df)*100,2),round(df.isna().sum()/len(df)*100,2)], axis = 1)
zeros_and_nulls.rename(columns={0: '% Zeros', 1: '% Nulls'}).sort_values(by = '% Nulls', ascending=False).head(30)

,% Zeros,% Nulls
TempAr,0.63,10.48
TempModulo,0.63,10.48
VelocidadeVento,2.04,10.48
E3TimeStamp,0.00,0.00
apparent_temperature_min_in_°C,0.00,0.00
direct_radiation_instant_in_W/m²,52.19,0.00
diffuse_radiation_instant_in_W/m²,50.48,0.00
direct_normal_irradiance_instant_in_W/m²,52.19,0.00
terrestrial_radiation_instant_in_W/m²,49.99,0.00
weathercode_in_wmo code_y,16.99,0.00


In [459]:
try:
  df = df.set_index('E3TimeStamp')
except:
  pass
try:
  del df['time_join_generation']
except:
  pass
try:
  del df['time_in_iso8601']
except:
  pass


df_descricao = pd.DataFrame()

df_descricao['No_None_Values_Count'] = round(df.apply(lambda x: x.count()),2)
df_descricao['None_Values_Count'] = round(df.apply(lambda x: x.isnull().sum()),2)
df_descricao['Zeroes_Values_Count'] = round(df.apply(lambda x: (x == 0).sum()),2)
df_descricao['Min_Values'] = round(df.apply(lambda x: x.min()),2)
df_descricao['Max_Values'] = round(df.apply(lambda x: x.max()),2)
df_descricao['Mean_Values'] = round(df.apply(lambda x: x.mean()),2)
df_descricao['Std_Values'] = round(df.apply(lambda x: x.std()),2)
df_descricao['Sum of Values'] = round(df.apply(lambda x: x.sum()),0)
df_descricao['Min_Date'] = df.index.min()
df_descricao['Max_Date'] = df.index.max()
df_descricao

,No_None_Values_Count,None_Values_Count,Zeroes_Values_Count,Min_Values,Max_Values,Mean_Values,Std_Values,Sum of Values,Min_Date,Max_Date
RadiacaoGlobal,17653,0,1982,-803.00,77479.00,11868.56,18198.24,209515760.0,2021-08-04 10:00:00,2023-08-09 22:00:00
yB,17653,0,10348,0.00,327501.10,279.95,3978.32,4942011.0,2021-08-04 10:00:00,2023-08-09 22:00:00
yC,17653,0,10435,0.00,331003.10,276.70,3986.86,4884528.0,2021-08-04 10:00:00,2023-08-09 22:00:00
yD,17653,0,11160,0.00,334239.90,250.39,3930.82,4420158.0,2021-08-04 10:00:00,2023-08-09 22:00:00
yE,17653,0,11102,0.00,15313.30,177.08,377.63,3125925.0,2021-08-04 10:00:00,2023-08-09 22:00:00
yF,17653,0,11096,0.00,12715.40,176.86,361.91,3122096.0,2021-08-04 10:00:00,2023-08-09 22:00:00
TempAr,15803,1850,111,0.00,179.67,23.78,6.79,375869.0,2021-08-04 10:00:00,2023-08-09 22:00:00
TempModulo,15803,1850,111,-10.00,182.48,26.59,15.58,420125.0,2021-08-04 10:00:00,2023-08-09 22:00:00
VelocidadeVento,15803,1850,360,-10.00,9.62,0.79,3.80,12492.0,2021-08-04 10:00:00,2023-08-09 22:00:00
temperature_2m_in_°C,17653,0,0,6.30,38.60,23.35,4.79,412231.0,2021-08-04 10:00:00,2023-08-09 22:00:00


In [477]:
df_generation[(pd.to_datetime(df_generation.E3TimeStamp, format = 'mixed') >= '2021-12-09') & (pd.to_datetime(df_generation.E3TimeStamp, format = 'mixed') <= '2021-12-09 18:00:00')]

,E3TimeStamp,PotenciaAtivaTotal,EnergiaTotal_DataLogger,EnergiaDiaria_DataLogger,RadiacaoGlobal,TempAr,TempModulo,VelocidadeVento,EnergiaTotal_LoteB,EnergiaDiaria_LoteB,...,EnergiaReativa_LoteC,EnergiaTotal_LoteD,EnergiaDiaria_LoteD,EnergiaReativa_LoteD,EnergiaTotal_LoteE,EnergiaDiaria_LoteE,EnergiaReativa_LoteE,EnergiaTotal_LoteF,EnergiaDiaria_LoteF,EnergiaReativa_LoteF
63416,2021-12-09 09:34:42.630000000,2883,1992737,5207,901.0,25.0,48.0,2.0,949063.2,1756.0,...,8.282,206490.6,1725.4,9.346,nan,nan,nan,nan,nan,nan
63417,2021-12-09 09:35:42.653000000,2889,1992786,5257,903.0,26.0,49.0,3.0,949079.8,1772.3,...,10.492,206506.9,1741.5,11.178,nan,nan,nan,nan,nan,nan
63418,2021-12-09 09:36:42.640000000,2897,1992835,5304,906.0,26.0,49.0,2.0,949095.8,1788.4,...,-3.314,206523.0,1757.5,-4.719,nan,nan,nan,nan,nan,nan
63419,2021-12-09 09:37:42.650000000,2904,1992883,5353,915.0,26.0,50.0,3.0,949111.6,1803.8,...,5.247,206539.1,1773.8,5.573,nan,nan,nan,nan,nan,nan
63420,2021-12-09 09:38:42.660000000,2914,1992930,5401,917.0,26.0,50.0,4.0,949126.8,1820.0,...,8.82,206555.2,1790.0,9.826,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192933,2021-12-09 17:55:24.557000000,436,2017072,28567,186.0,31.0,32.0,1.0,953885.3,6577.9,...,2.931,211501.3,6735.9,3.587,7796.2,6820.7,3.184,1766.6,1766.6,3.094
192934,2021-12-09 17:56:24.563000000,420,2017079,28574,182.0,31.0,31.0,1.0,953886.6,6579.2,...,3.151,211502.9,6737.5,3.804,7797.6,6822.1,3.462,1767.8,1767.8,3.319
192935,2021-12-09 17:57:24.560000000,405,2017085,28580,178.0,31.0,31.0,1.0,953887.8,6580.4,...,2.919,211504.4,6739.0,3.588,7799.0,6823.5,3.199,1769.1,1769.1,3.12
192936,2021-12-09 17:58:24.563000000,390,2017092,28587,173.0,31.0,31.0,2.0,953889.0,6581.6,...,2.882,211505.8,6740.4,3.559,7800.5,6825.0,3.168,1770.3,1770.3,3.064


In [476]:
pd.to_datetime(df_generation.E3TimeStamp, format = 'mixed')

0        2021-08-17 07:42:20.090
1        2021-08-17 07:43:20.043
2        2021-08-17 07:44:20.037
3        2021-08-17 07:45:20.127
4        2021-08-17 07:46:20.027
                   ...          
949868   2023-08-09 22:16:10.357
949869   2023-08-09 22:17:10.407
949870   2023-08-09 22:18:10.380
949871   2023-08-09 22:19:10.373
949872   2023-08-09 22:20:10.317
Name: E3TimeStamp, Length: 949873, dtype: datetime64[ns]